In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule
import numpy as np

In [3]:
data = torch.load('./pattern-toys-noise-50.pth')
print(len(data))

19412


In [4]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
from sequential.seq2pat import Seq2Pat, Attribute

# data = [
#     [1,2,112,2223,3,134],
#     [1,2,6456,324,1521345,123],
#     [1,2,125,4365,23421,203985],
#     [2,3,12358,4532,23541,38954],
#     [2,3,353,4234,45453,4678],
#     [2,3,234,3342,11234,465],
# ]

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=data, n_jobs=2, max_span=10)
# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=2)
patterns_value = [_[:-1] for _ in patterns]
patterns_count = [_[-1] for _ in patterns]
# print(patterns)
print(len(patterns))

42384


In [69]:
original_train = torch.load('./dataset/amazon-toys-seq-noise-50/toy/train_ori.pth')
len(original_train)

109361

In [76]:
test = [torch.tensor(_) for _ in patterns_value]
test = pad_sequence(test, batch_first=True)
seq_len = test.bool().sum(-1)
patterns_value_copy = deepcopy(patterns_value)
while(True):
    test = [torch.tensor(_) for _ in patterns_value_copy]
    test = pad_sequence(test, batch_first=True)
    seq_len = test.bool().sum(-1)
    new_patterns = []
    for idx, _ in enumerate(test):
        last_item = _[seq_len[idx] - 1]
        succesor_list = torch.where(test[:, 0] == last_item)[0]
        if len(succesor_list) > 0:
            for succesor in succesor_list:
                new_patterns.append(
                    patterns_value_copy[idx] + \
                    patterns_value_copy[succesor][1:]
                )
        else:
            new_patterns.append(
                _
            )
    print(len(new_patterns))
    if len(new_patterns) == len(patterns_value_copy):
        break
    patterns_value = new_patterns


KeyboardInterrupt: 

In [70]:
max_seq_len = 50
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:]
    else:
        return seq + [0] * (max_seq_len - cur_seq_len)


train_set = set()

for pattern in patterns:
    seq = pattern[:-1]
    seq_len = len(seq)
    for _ in range(1, seq_len):
        train_set.add(tuple(
            truncate_or_pad(seq[:_]) + [seq[_]],
        ))

In [71]:
train_list = []
for _ in train_set:
    train_list.append([
        0,
        list(_)[:-1],
        list(_)[-1],
        sum([a != 0 for a in list(_)[:-1]]),
        1,
        0
    ])
print(len(train_list))
print(len(original_train))

42384
109361


In [38]:
np.random.choice(np.arange(5), 1)

array([4])

In [72]:
semantic_augmented_train_list = []
all_target_item = np.array([_[2] for _ in train_list])
seq_array = np.array([_[1] for _ in train_list])
seq_len = np.array([_[3] for _ in train_list])
all_idx = np.arange(len(train_list))
for idx, _ in enumerate(train_list):
    real_seq, target_item = _[1][:_[3]], _[2]
    match_idx = (all_target_item == target_item) & (all_idx != idx)
    match_seq = seq_array[match_idx]
    match_seq_len = seq_len[match_idx]
    if len(match_seq) > 0:
        selected_idx = np.random.choice(np.arange(len(match_seq)), 1)[0]
        selected_seq = match_seq[selected_idx][:match_seq_len[selected_idx]].tolist()
        new_seq = truncate_or_pad(selected_seq + real_seq)
        semantic_augmented_train_list.append([
            0,
            new_seq,
            target_item,
            sum([a != 0 for a in list(new_seq)]),
            1,
            0
        ])
    else:
        semantic_augmented_train_list.append(_)

In [22]:
torch.save(train_list + original_train, 'train_new.pth')

In [75]:
torch.save(semantic_augmented_train_list + original_train, 'train_new-sem.pth')

In [ ]:
torch.save(train_list, 'train_new-pure.pth')